In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -U transformers
!pip install -U accelerate
!pip install -U datasets

# Load and Prepare Dataset

In [3]:
import pandas as pd
import json
import requests

In [4]:
from datasets import load_dataset

ds = load_dataset("tner/mit_movie_trivia")

In [5]:
ds

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 6816
    })
    validation: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['tokens', 'tags'],
        num_rows: 1953
    })
})

In [6]:
tag2id = requests.get(f"https://huggingface.co/datasets/tner/mit_movie_trivia/raw/main/dataset/label.json").json()
print(tag2id)

{'O': 0, 'B-Actor': 1, 'I-Actor': 2, 'B-Plot': 3, 'I-Plot': 4, 'B-Opinion': 5, 'I-Opinion': 6, 'B-Award': 7, 'I-Award': 8, 'B-Year': 9, 'B-Genre': 10, 'B-Origin': 11, 'I-Origin': 12, 'B-Director': 13, 'I-Director': 14, 'I-Genre': 15, 'I-Year': 16, 'B-Soundtrack': 17, 'I-Soundtrack': 18, 'B-Relationship': 19, 'I-Relationship': 20, 'B-Character_Name': 21, 'I-Character_Name': 22, 'B-Quote': 23, 'I-Quote': 24}


In [7]:
id2tag = {}
for key, value in tag2id.items():
    id2tag[value] = key
print(id2tag)

{0: 'O', 1: 'B-Actor', 2: 'I-Actor', 3: 'B-Plot', 4: 'I-Plot', 5: 'B-Opinion', 6: 'I-Opinion', 7: 'B-Award', 8: 'I-Award', 9: 'B-Year', 10: 'B-Genre', 11: 'B-Origin', 12: 'I-Origin', 13: 'B-Director', 14: 'I-Director', 15: 'I-Genre', 16: 'I-Year', 17: 'B-Soundtrack', 18: 'I-Soundtrack', 19: 'B-Relationship', 20: 'I-Relationship', 21: 'B-Character_Name', 22: 'I-Character_Name', 23: 'B-Quote', 24: 'I-Quote'}


In [8]:
print(ds['train'][0]['tokens'])
print(ds['train'][0]['tags'])

['what', '1995', 'romantic', 'comedy', 'film', 'starred', 'michael', 'douglas', 'as', 'a', 'u', 's', 'head', 'of', 'state', 'looking', 'for', 'love']
[0, 9, 10, 15, 0, 0, 1, 2, 0, 3, 4, 4, 4, 4, 4, 4, 4, 4]


# Model Building

In [9]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
input = ds['train'][66]
print(input['tokens'])
print(input['tags'])

['a', '2009', 'animated', 'movie', 'by', '20', 'th', 'century', 'fox', 'starring', 'ray', 'romano', 'john', 'leguizamo', 'and', 'denis', 'leary']
[0, 9, 10, 15, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 1, 2]


In [11]:
output = tokenizer(input['tokens'], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(output['input_ids'])
print(tokens)

['[CLS]', 'a', '2009', 'animated', 'movie', 'by', '20', 'th', 'century', 'fox', 'starring', 'ray', 'romano', 'john', 'leg', '##ui', '##zam', '##o', 'and', 'denis', 'leary', '[SEP]']


In [12]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []

        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)

            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [13]:
tokenized_dataset = ds.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [14]:
print(tokenized_dataset['train'][66])

{'tokens': ['a', '2009', 'animated', 'movie', 'by', '20', 'th', 'century', 'fox', 'starring', 'ray', 'romano', 'john', 'leguizamo', 'and', 'denis', 'leary'], 'tags': [0, 9, 10, 15, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 1, 2], 'input_ids': [101, 1037, 2268, 6579, 3185, 2011, 2322, 16215, 2301, 4419, 4626, 4097, 22070, 2198, 4190, 10179, 20722, 2080, 1998, 11064, 22986, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, 0, 9, 10, 15, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, -100, -100, -100, 0, 1, 2, -100]}


# Data Collatior and Metrics

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [16]:
!pip install evaluate
!pip install seqeval

In [17]:
import evaluate
import numpy as np

metric = evaluate.load("seqeval")

label_names = list(tag2id)

def compute_metrics(eval_preds):
    logits, labels = eval_preds

    predictions = np.argmax(logits, axis=-1)
    true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

    true_predictions = [[label_names[p] for p, l in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# Model Training

In [18]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [19]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2tag,
    label2id=tag2id,
).to(device)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    output_dir="model_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=[],
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [21]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.690200,0.390142,0.622207,0.684519,0.651878,0.876087
2,0.319800,0.356558,0.638936,0.695891,0.666198,0.879525
3,0.268700,0.361914,0.646190,0.706163,0.674847,0.882275


TrainOutput(global_step=2556, training_loss=0.38709949476991273, metrics={'train_runtime': 190.3761, 'train_samples_per_second': 107.408, 'train_steps_per_second': 13.426, 'total_flos': 176003687459856.0, 'train_loss': 0.38709949476991273, 'epoch': 3.0})

In [22]:
trainer.save_model("ner_distilbert")

# Prediction and Load & Save Model

In [26]:
from transformers import pipeline

checkpoint = "ner_distilbert"
pipe = pipeline(
    "token-classification", model=checkpoint, aggregation_strategy="simple"
)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [39]:
import spacy
from spacy import displacy

nlp = spacy.blank("en")

text = (
"""
Today, Dhaka's Star Cineplex released two Hollywood movies, "Venom: The Last Dance" and "Smile 2", as announced in a press release from the multiplex.
"Venom: The Last Dance" is the latest installment in the "Venom" franchise. It is the third and final chapter, and the movie is being released worldwide today.
Google News LinkFor all latest news, follow The Daily Star's Google News channel.

Audiences in Bangladesh can watch it at Star Cineplex on its release day. Tom Hardy reprises his role as Venom, and the cast also includes Juno Temple, Chiwetel Ejiofor, and Clarke Backo.

On the other hand, "Smile 2" is a film in the psycho-horror genre. Released internationally on October 16, it has already attracted audiences. The film is directed by Parker Finn and features stars like Sosie Bacon, Peter Jacobson, Kyle Gallner, and Lucas Gage in various roles—all of whom starred in the first instalment.
"""
)

ner_results = pipe(text)
doc = nlp(text)
ents = []

for result in ner_results:
    # Create a SpaCy-compatible entity dictionary
    start, end, label = result["start"], result["end"], result["entity_group"]
    span = doc.char_span(start, end, label=label)

    if span is not None:  # Only add valid spans
        ents.append(span)

# Set entities to the Doc object
doc.ents = ents

displacy.render(doc, style="ent", jupyter=True)